### Elasticsearch Restoring Emotion Classifier/Crawling
Here we are going to resotre the emotion classifier that we trained in the previous chapter of the series. Then we will crawl real-time tweets from Twitter to classify, and then index into Elasticsearch. Then we are going to use Kibana to analyze the predictions and see where the model is doing well and not so well. In fact, we are going to use the inference of the model, to answer a few interesting questions using Kibana powerful analytic functionalities.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics, decomposition, pipeline, dummy
import torch
import torch.nn.functional as F
import torch.nn as nn
import os
import matplotlib.pyplot as plt
%matplotlib inline
import helpers.pickle_helpers as ph
import time
import math
from sklearn.cross_validation import train_test_split
import re

### Parameters
Let's define the hyperparameters again since we need then when restoring the model

In [ ]:
NUM_WORDS = 10000 # max size of vocabulary
EMBEDDING_DIM = 128
HIDDEN_SIZE = 256
ATTENTION_SIZE = 150
KEEP_PROB = 0.8
BATCH_SIZE = 128
NUM_EPOCHS = 50 # Model easily overfits without pre-trained words embeddings, that's why train for a few epochs
DELTA = 0.5
NUM_LAYERS = 3
LEARNING_RATE = 0.001

### Import Embeddings
Also, we will need the word embeddings to perform classification.

In [ ]:
### load word embeddings and accompanying vocabulary
wv = ph.load_from_pickle("data/hashtags_word_embeddings/es_py_cbow_embeddings.p")
vocab = ph.load_from_pickle("data/hashtags_word_embeddings/es_py_cbow_dictionary.p")

### Redefine Model
Since we used a very a naive way to store our model in the previous chapter, we will need to redefine the same model again. With the latest version of PyTorch there better ways to store and restore models without having to do all of this unecessary steps. For now, let's just use this simple approach to restore our models.

In [ ]:
class EmoNet(nn.Module):
    def __init__(self, num_layers, hidden_size, embedding_dim, output_size, dropout):
        super(EmoNet, self).__init__()
        self.embedding_dim = embedding_dim
        self.keep_prob = dropout
        self.hidden_size = hidden_size
        self.nlayers = num_layers
        self.output = output_size
        
        self.dropout  = nn.Dropout(p=self.keep_prob)
        
        self.rnn = nn.LSTM(input_size=self.embedding_dim,
                                 hidden_size=self.hidden_size, 
                                 num_layers=self.nlayers,
                                 dropout=self.keep_prob)
        self.linear = nn.Linear(self.hidden_size, output_size)
        
    def forward(self, inputs):
        # batch_size X seq_len X embedding_dim -> seq_len, batch_size, embedding_dim
        X = inputs.permute(1,0,2)
        self.rnn.flatten_parameters()
        output, hidden = self.rnn(X)
        (_, last_state) = hidden      
        out = self.dropout(output[-1])  
        out = self.linear(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs        

In [ ]:
### restoring the model
tmodel = torch.load('model/elastic_hashtag_model/emonet')

In [ ]:
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

### Helper Function
For simplicity, let's redefine the helper function we used before. If you want to further optimize your code, you could easily put these reusable functions into a seperate library.

In [ ]:
### TODO: move this preprocessing helper functions
def clearstring(string):
    string = string.lower()
    string = re.sub('[^\'\"A-Za-z0-9 ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    string = [y for y in string if len(y) > 3 and y.find('nbsp') < 0]
    return string

def generate_embeds_with_pads(tokens, max_size):
   
    padded_embedding = []
    for i in range(max_size):
        if i+1 > len(tokens): # do padding
            padded_embedding.append(list(np.zeros(EMBEDDING_DIM)))
        else: # do embedding for existing tokens
            padded_embedding.append(list(wv[vocab[tokens[i]]]))  
    return padded_embedding

def remove_unknown_words(tokens):
    return [t for t in tokens if t in vocab]

### Input Transformations
When we are crawling data from the Twitter API, we need to preprocess it and then transform the input to word embedding representations. Same process as used for the classifier int the previous chapter, just that this case we are using it to classify real-time data.

In [ ]:
def transform_data_to_input(text):
    """ Accepts only one text as input; can be done by batches later on"""
    ### TODO Do the preprocessing here
    text = clearstring(text) # list of tokens
    text = remove_unknown_words(text)
    emb = generate_embeds_with_pads(text, len(text))
    
    return emb

emo_map = {0: 'anger', 
           1: 'anticipation', 
           2: 'disgust', 
           3: 'fear', 
           4: 'joy', 
           5: 'sadness',
           6: 'surprise',
           7: 'trust'}

### Sample Classification of text
Let's test to see if those function above work on a dummy text. Wow! You can see that the classifier classifies the word "unhappy" to sadness, which means the model is good to some extent.

In [ ]:
### Get the emotion from tweet
x = transform_data_to_input("unhappy") # put tweet here
final_x = torch.FloatTensor(np.array(x))
final_x = final_x.unsqueeze(0)
emo_map[torch.argmax(tmodel(final_x.to(device))).detach().item()]

### Crawl Data And Index to Elasticsearch
Now to the main part of this series. We have pretrained embeddings, we have trained and stored a classifier, but the best part of all will happen next. We will crawl real-time tweets and classify them into an emotion. We will then store those inferences of the model, along with the text, into Elasticsearch. We will then connect the Elasticsearch with Kibana and analyze our results. 

In [ ]:
### import a few useful libraries
import crawlers.config as cf
from elasticsearch import Elasticsearch
from elasticsearch import helpers
es = Elasticsearch(cf.ELASTICSEARCH['hostname'])
import sys, json
import crawlers.config as config
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from elasticsearch import helpers
from elasticsearch import Elasticsearch
import re
from copy import deepcopy

### Crawler Configurations
The following are just some extra configurations that are needed for the crawler. Keep in mind that these configurations are mostly obtained from the config library provided with the repository.

In [ ]:
LANGUAGES = ['en']
WANTED_KEYS = [
    'id_str',
    'text',
    'created_at',
    'in_reply_to_status_id_str',
    'in_reply_to_user_id_str',
    'retweeted',
    'entities']  # Wanted keys to store in the database
KEYWORDS = config.KEYWORDS['joy'] + \
           config.KEYWORDS['trust'] + \
           config.KEYWORDS['fear'] + \
           config.KEYWORDS['surprise'] + \
           config.KEYWORDS['sadness'] + \
           config.KEYWORDS['disgust'] + \
           config.KEYWORDS['anger'] + \
           config.KEYWORDS['anticipation'] + \
           config.KEYWORDS['other']

print(len(KEYWORDS))

### Helper functions
Below are a few helper functions which will be useful for the crawler in order to properly store the information we want on Elasticsearch. Again, the notebook could be simplified by putting this code in a seperate Python file. For now, we will stick to our long functions just to have everything in one place, where I can easily explain the components of the tutorial.

In [ ]:
def convert_to_es_format(tweet):
    """Convert into elastic format"""
    action = [
        {
            "_index": cf.ELASTICSEARCH['index'],
            "_type": cf.ELASTICSEARCH['type'],
            "_source": {
                "emotion": tweet["emotion"],
                "created_at": tweet["created_at"],
                "tweet_id": tweet["tweet_id"],
                "text": tweet["text"],
                "hashtags": tweet["hashtags"]                 
            }          
        }
    ]
    return action

def post_tweet_to_es(doc):
    """ insert into Elasticsearch in bulk """
    helpers.bulk(es, doc)

def get_hashtags(list):
    """obtain hashtags from tweet"""
    hashtags = []
    for h in list:
        hashtags.append(h['text'])
    return hashtags

def predict_emotion(text):
    """ ouput prediction of the model """
    x = transform_data_to_input(text) # put tweet here
    final_x = torch.FloatTensor(np.array(x))
    final_x = final_x.unsqueeze(0)
    return emo_map[torch.argmax(tmodel(final_x.to(device))).detach().item()]

def format_to_print(tweet, hashtags):
    """ format raw tweet """
    tweet_dict = {'text':tweet['text'],
            'created_at': tweet['created_at'],
            'tweet_id': tweet['id_str'],
            'emotion': predict_emotion(tweet['text']),
            'hashtags': hashtags}
    return tweet_dict

### The Crawler
And we are finally ready to start crawling and storing our data. The crawler code below is standard code for crawling from the Twitter API. You will need to configure all your tokens in the config file so that this code can work. The `on_data` function in the class below achives everything we want: from preprocessing it, to classifying it, to indexing it into Elasticsearch. Spend some time analyzing the code below and make sure you understand how it is doing everything which I just explained. 

In [ ]:
# Stream Listener
class Listener(StreamListener):

    @staticmethod
    def on_data(data):
        try:
            reponse = json.loads(data)
            tweet = {key: reponse[key] for key in set(WANTED_KEYS) & set(reponse.keys())}

            hashtags = get_hashtags(tweet['entities']['hashtags'])
            anyretweet= re.findall(r'RT|https|http', str(tweet['text']))

            ### formatting tweet
            final_tweet = format_to_print(tweet, hashtags)

            ### make insertions
            if not anyretweet:                                
                f = deepcopy(final_tweet)
                
                ### insert to elasticsearch
                es_final_tweet = convert_to_es_format(f)
                post_tweet_to_es(es_final_tweet)

        except Exception as e:
            print(e)
            #print ("--------------On data function------------")
            return True

    @staticmethod
    def on_error(status):
        print ("--------------On error function------------")
        print (status)
        return True

    @staticmethod
    def on_timeout():
        print ("--------------On timeout function------------")
        print >> sys.stderr, 'Timeout...'
        return True  # Don't kill the stream

    @staticmethod
    def on_status(status):
        print ("--------------On status function------------")
        print (status.text)
        return True

### Start Crawling...
And for the moment you have been waiting for. Let's start the crawler! You can let the crawler run for as much time as you want. Since this is a crawler, I do suggest you convert this notebook into a Python script to make it more efficient. You will also notice sometimes that the crawler will output a warning "cannot unsqueeze empty tensor", you can ignore it since sometimes tweets are too short to deduce any information from them and so the classifier won't be able to infer anything from it. 

In [ ]:
### Starts streaming
while True:
    try:
        auth = OAuthHandler(
            config.TWITTER['consumer_key'], config.TWITTER['consumer_secret'])
        auth.set_access_token(
            config.TWITTER['access_token'], config.TWITTER['access_secret'])
        print("Crawling, Classifying, and Indexing tweets...")
        twitterStream = Stream(auth, Listener())
        twitterStream.filter(languages=LANGUAGES, track=KEYWORDS)
    except KeyboardInterrupt:
        print ("--------------On keyboard interruption function------------")
        print("Bye")
        sys.exit()